In [8]:
from functions import *

# ITU Data to TSV

In [9]:
itu_df = pd.read_csv(r"../data/public_data/train.csv", index_col="id")
itu_df.head()

,language,sentence,tokenized_sentence,expression,category,indices
id,,,,,,
1,tr,"Ödevler, Sınavlar, Evet yaşanması gereken zama...","['Ödevler', ',', 'Sınavlar', ',', 'Evet', 'yaş...",zaman öldürmek,idiomatic,"[7, 8]"
2,tr,Belediye bugün yolları açtı.,"['Belediye', 'bugün', 'yolları', 'açtı', '.']",yol açmak,literal,[-1]
3,tr,Planı ortaya çıkmasın diye elinden geleni yapmış.,"['Planı', 'ortaya', 'çıkmasın', 'diye', 'elind...",ortaya çıkmak,idiomatic,"[1, 2]"
4,tr,Aldığın bunca tedbire rağmen halen üstüne geli...,"['Aldığın', 'bunca', 'tedbire', 'rağmen', 'hal...",üstüne almak,literal,[-1]
5,tr,"Yazdıkların yanlış olmuş, istersen defterden s...","['Yazdıkların', 'yanlış', 'olmuş', ',', 'ister...",defterden silmek,literal,[-1]


In [10]:
train_path = r"../data/public_data/train.csv"
dev_path = r"../data/public_data/eval.csv"

itu_to_tsv(train_path, r"./ITU/train.tsv")
itu_to_tsv(dev_path, r"./ITU/dev.tsv")


# PARSAME Data to TSV

In [11]:
cupt_data = parse_cupt(r"../data/PARSAME 1.1 data/sharedtask-1.1-TR/train.cupt")
for word in cupt_data[1]:
    print(word)

['1', 'Türkiye', 'Türkiye', 'Noun', 'Prop', 'A3sg|Nom|Pnon', '17', 'SUBJECT', '_', '_', '*']
['2', "'de", "'de", '?', '_', '_', '3', 'MODIFIER', '_', '_', '*']
['3', 'bankaların', 'banka', 'Noun', '_', 'A3pl|Gen|Pnon', '6', 'POSSESSOR', '_', '_', '*']
['4', 'bir', 'bir', 'Adj', 'Num', '_', '6', 'DETERMINER', '_', '_', '*']
['5', 'siyasi', 'siyasi', 'Adj', '_', '_', '6', 'MODIFIER', '_', '_', '*']
['6', 'gücü', 'güç', 'Noun', 'NAdj', 'A3sg|Nom|P3sg', '8', 'SUBJECT', '_', '_', '*']
['7', 'de', 'de', 'Conj', '_', '_', '6', 'INTENSIFIER', '_', '_', '*']
['8', 'var', 'var', 'Adj', '_', '_', '17', 'COORDINATION', '_', '_', '*']
['9', ',', ',', 'Punc', '_', '_', '8', 'PUNCTUATION', '_', '_', '*']
['10', 'bu', 'bu', 'Det', '_', '_', '11', 'DETERMINER', '_', '_', '*']
['11', 'nedenle', 'neden', 'Noun', '_', 'A3sg|Ins|Pnon', '13', 'MODIFIER', '_', '_', '*']
['12', 'ilerde', 'ilerd', 'Noun', '_', 'A3sg|Dat|Pnon', '13', 'MODIFIER', '_', '_', '*']
['13', '_', 'örgütle', 'Verb', '_', 'Pass|Pos', '14

In [12]:
# Ensure output folders exist
tr_out_path = r"../data/PARSAME 1.1 data/transformed-TR/"
it_out_path = r"../data/PARSAME 1.1 data/transformed-IT/"
os.makedirs(tr_out_path, exist_ok=True)
os.makedirs(it_out_path, exist_ok=True)

tr_in_path = r"../data/PARSAME 1.1 data/sharedtask-1.1-TR/"
it_in_path = r"../data/PARSAME 1.1 data/sharedtask-1.1-IT/"

# Transform Turkish
tranform_cupt_to_tsv(tr_in_path+"train.cupt", tr_out_path+"train.tsv", "tr")
tranform_cupt_to_tsv(tr_in_path+"dev.cupt", tr_out_path+"dev.tsv", "tr")
tranform_cupt_to_tsv(tr_in_path+"test.cupt", tr_out_path+"test.tsv", "tr")

# Transform Italian
tranform_cupt_to_tsv(it_in_path+"train.cupt", it_out_path+"train.tsv", "it")
tranform_cupt_to_tsv(it_in_path+"dev.cupt", it_out_path+"dev.tsv", "it")
tranform_cupt_to_tsv(it_in_path+"test.cupt", it_out_path+"test.tsv", "it")

In [13]:
# Create output folder
parsame_output_path = r"./PARSAME/"
os.makedirs(parsame_output_path, exist_ok=True)

# Combine train, dev, test TSVs
combine_tsv_files(tr_out_path+"train.tsv", it_out_path+"train.tsv", parsame_output_path+"train.tsv")
combine_tsv_files(tr_out_path+"dev.tsv", it_out_path+"dev.tsv", parsame_output_path+"dev.tsv")
combine_tsv_files(tr_out_path+"test.tsv", it_out_path+"test.tsv", parsame_output_path+"test.tsv")

# ID10M

ID10M is alredy in tsv format we need to add language.

In [14]:
id_input_path = r"../data/ID10M/"
# check outpath
out_path = r"./ID10M/"
os.makedirs(out_path, exist_ok=True)
# Add language to ID10M files
add_language(id_input_path+"train.tsv", out_path+"train.tsv", "it")
add_language(id_input_path+"dev.tsv", out_path+"dev.tsv", "it")
add_language(id_input_path+"test.tsv", out_path+"test.tsv", "it")